# 🚀 LangChain + Arize Observability Demo

## Overview
This notebook demonstrates how to integrate **LangChain** with **Arize** for comprehensive AI observability and monitoring. We'll build a multi-step AI workflow and trace every interaction for debugging and optimization. - Created by Ayyanar Jeyakrishnan

### What You'll Learn:
- ✅ Setting up Arize tracing for LangChain applications
- ✅ Building sequential AI workflows with LangChain LCEL (LangChain Expression Language)
- ✅ Monitoring AI performance and debugging with Arize
- ✅ Best practices for production AI observability

### Use Case:
We'll create a **Play Review Generator** that:
1. Takes a play title as input
2. Generates a synopsis using GPT-4
3. Creates a professional review based on the synopsis
4. Traces every step for monitoring and optimization

## 📦 Step 1: Install Dependencies

First, we install all required packages for LangChain, OpenAI integration, and Arize observability.

In [1]:
# Core LangChain packages
!pip install -q langchain langchain_community langchain-openai

# Arize observability packages
!pip install -q openinference-instrumentation-openai openai
!pip install -q arize-otel
!pip install -q "arize[AutoEmbeddings]" "openinference-instrumentation-langchain>=0.1.4"

print("✅ All packages installed successfully!")

✅ All packages installed successfully!


## 🔧 Step 2: Configure Arize Observability

Here we set up **Arize** to automatically trace all LangChain operations. This gives us:
- **Real-time monitoring** of AI model calls
- **Performance metrics** (latency, token usage, costs)
- **Error tracking** and debugging capabilities
- **Chain visualization** to understand workflow execution

In [2]:
# Import Arize observability components
from arize.otel import register
from getpass import getpass

# Configure Arize tracing
tracer_provider = register(
    space_id=getpass("🔑 Enter your Arize Space ID: "),
    api_key=getpass("🔑 Enter your Arize API Key: "),
    project_name="langchain-arize-demo",  # This will appear in your Arize dashboard
)

# Enable automatic LangChain instrumentation
from openinference.instrumentation.langchain import LangChainInstrumentor
LangChainInstrumentor().instrument(tracer_provider=tracer_provider)

print("🔭 Arize observability configured successfully!")
print("📊 All LangChain operations will now be automatically traced")

🔭 OpenTelemetry Tracing Details 🔭
|  Arize Project: langchain-arize-demo
|  Span Processor: BatchSpanProcessor
|  Collector Endpoint: otlp.arize.com
|  Transport: gRPC
|  Transport Headers: {'authorization': '****', 'api_key': '****', 'arize-space-id': '****', 'space_id': '****', 'arize-interface': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.

🔭 Arize observability configured successfully!
📊 All LangChain operations will now be automatically traced


Failed to export traces to otlp.arize.com, error code: StatusCode.INTERNAL
Failed to export traces to otlp.arize.com, error code: StatusCode.INTERNAL
Failed to export traces to otlp.arize.com, error code: StatusCode.INTERNAL
Failed to export traces to otlp.arize.com, error code: StatusCode.INTERNAL
Failed to export traces to otlp.arize.com, error code: StatusCode.INTERNAL
Failed to export traces to otlp.arize.com, error code: StatusCode.INTERNAL
Failed to export traces to otlp.arize.com, error code: StatusCode.INTERNAL
Failed to export traces to otlp.arize.com, error code: StatusCode.INTERNAL
Failed to export traces to otlp.arize.com, error code: StatusCode.INTERNAL


## 🤖 Step 3: Initialize AI Components

We set up our OpenAI model configuration and create the base LLM that will power our application.

In [3]:
import os
import getpass

# Get OpenAI API key securely
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or getpass.getpass("🔑 OpenAI API Key: ")

# Configuration class for easy model management
class Config:
    OPENAI_MODEL = "gpt-4o-mini"  # Fast, cost-effective model for demos
    TEMPERATURE = 0  # Deterministic outputs for consistent results

print(f"🎯 Using model: {Config.OPENAI_MODEL}")
print(f"🌡️ Temperature: {Config.TEMPERATURE} (deterministic)")

🎯 Using model: gpt-4o-mini
🌡️ Temperature: 0 (deterministic)


## 🧪 Step 4: Test Basic LLM Functionality

Let's verify our setup works with a simple translation example. This will also generate our first trace in Arize!

In [4]:
from langchain_openai import ChatOpenAI

# Initialize the ChatOpenAI model
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY, 
    model=Config.OPENAI_MODEL, 
    temperature=Config.TEMPERATURE
)

# Test with a simple translation task
messages = [
    ("system", "You are a helpful assistant that translates English to French."),
    ("human", "I love programming."),
]

# This call will be automatically traced by Arize!
response = llm.invoke(messages)

print("🔤 Translation Result:")
print(f"📝 {response.content}")
print("\n✅ First trace sent to Arize! Check your dashboard.")

/Users/jayyanar/Downloads/evidence-auditor/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔤 Translation Result:
📝 J'aime la programmation.

✅ First trace sent to Arize! Check your dashboard.


## 🎭 Step 5: Build the Play Review Generator

Now we'll create a more complex workflow using **LangChain Expression Language (LCEL)**. This demonstrates:
- **Sequential processing**: Output of one step becomes input of the next
- **Prompt templates**: Reusable, parameterized prompts
- **Chain composition**: Building complex workflows from simple components

In [16]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain_core.output_parsers import StrOutputParser

# 1. Initialize the modern chat model
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model=Config.OPENAI_MODEL, temperature=0)

# 2. Define the first prompt (synopsis generation)
synopsis_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a playwright. Given a play title, write a synopsis."),
    HumanMessage(content="Title: {title}")
])

# 3. Define the second prompt (review generation)
review_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a New York Times critic. Write a review for a play based on its synopsis."),
    HumanMessage(content="Play Synopsis:\n{synopsis}")
])


In [17]:
# 4. Create individual steps
synopsis_chain = synopsis_prompt | llm | StrOutputParser()
review_chain = review_prompt | llm | StrOutputParser()

In [18]:
# 5. Create full sequential chain using Runnables
full_chain = (
    {"title": lambda x: x["input"]} 
    | synopsis_chain 
    | {"synopsis": lambda x: x} 
    | review_chain
)

In [19]:
# 6. Test Inputs
test_inputs = [
    {"input": "documentary about pandas who are about to be extinct because of global warming"},
    {"input": "once upon a time in hollywood"},
    {"input": "the best mo observability tooling"},
]

In [20]:
# 7. Run through chain and print results
for inp in test_inputs:
    result = full_chain.invoke(inp)
    print(f"\n📘 Input: {inp['input']}\n📝 Review: {result}\n")


📘 Input: documentary about pandas who are about to be extinct because of global warming
📝 Review: Title: "Echoes of Tomorrow"

In the heart of a bustling metropolis, "Echoes of Tomorrow" unfolds as a poignant exploration of memory, loss, and the indelible marks left by those we love. The play, written by the talented newcomer Clara Jensen, weaves together the lives of three characters: Maya, a young woman grappling with the recent death of her mother; Sam, her estranged father, who is haunted by his own regrets; and Lila, a mysterious figure from Maya's past who appears to guide her through her grief.

Set against a backdrop of shifting timelines, the narrative oscillates between the present and flashbacks that reveal the complexities of familial relationships. Jensen's writing is both lyrical and raw, capturing the essence of human emotion with a deft hand. The dialogue crackles with authenticity, and the characters are richly drawn, each embodying a different facet of grief and heal

## ⛓️ Step 6: Create Individual Chain Components

We build the individual processing steps using the **pipe operator (|)** - LangChain's powerful composition syntax.

## 🚀 Next Steps & Best Practices

### 🔧 **Production Recommendations**

1. **Environment Management**
   ```python
   # Use different projects for different environments
   project_name = f"my-app-{os.getenv('ENVIRONMENT', 'dev')}"
   ```

2. **Error Handling**
   ```python
   # Add retry logic and fallbacks
   from langchain.callbacks import get_openai_callback
   ```

3. **Cost Monitoring**
   ```python
   # Track token usage for cost optimization
   with get_openai_callback() as cb:
       result = chain.invoke(input)
       print(f"Cost: ${cb.total_cost}")
   ```

### 📚 **Additional Resources**
- [LangChain Documentation](https://python.langchain.com/)
- [Arize AI Documentation](https://docs.arize.com/)
- [OpenInference Instrumentation](https://github.com/Arize-ai/openinference)

### 🎉 **Congratulations!**
You've successfully built and monitored a production-ready AI workflow with LangChain and Arize observability!